In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#!ls "/content/gdrive/MyDrive/"

In [3]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 7.1MB/s 
     |████████████████████████████████| 174kB 54.3MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 71kB 12.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack,L2BasicIterativeAttack,L2ProjectedGradientDescentAttack

class Model_Drop(nn.Module):

    def __init__(self):
        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

        self.drop_layer = nn.Dropout(p=0.5)
        self.drop_layer_conv = nn.Dropout2d(p=0.5)

    def last_hidden_layer_output(self, x):

        x = F.relu(self.conv1(x),inplace=True)
        x = F.max_pool2d(F.relu(self.conv2(x),inplace=True), kernel_size=2, stride=2)

        x = F.relu(self.conv3(x),inplace=True)
        x = F.max_pool2d(F.relu(self.conv4(x),inplace=True), kernel_size=2, stride=2)

        x = F.relu(self.conv5(x),inplace=True)
        x = self.drop_layer_conv(F.max_pool2d(F.relu(self.conv6(x),inplace=True), kernel_size=2, stride=2))

        x = x.view(-1, 4096)
        x = self.drop_layer(F.relu(self.fc1(x)))
        x = self.drop_layer(F.relu(self.fc2(x)))
        return x

    def forward(self, x):

        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x

train_data = datasets.CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

torch.manual_seed(2)
torch.cuda.manual_seed(2)

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

criterion = nn.CrossEntropyLoss()

T = 100

def epoch(loader, model, opt=None):

    if opt:
        model.train()
    else:
        model.eval()

    total_loss, total_err = 0., 0.

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        
        outputs = model(X)

        log_prob_s = F.log_softmax(outputs / T, dim=1)

        loss = F.nll_loss(log_prob_s,y)
        #loss = criterion(log_prob_s, y)

        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_err += (outputs.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)




Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [5]:
model_cnn = Model_Drop()
model_cnn.to(device)

opt = optim.Adam(model_cnn.parameters(), lr=0.001)
#opt = optim.SGD(model_cnn.parameters(), lr=0.1, momentum=0.5)

for te in range(50):

    #if te == 4:
      #for param_group in opt.param_groups:
        #param_group["lr"] = 1e-3

    train_err, train_loss = epoch(train_loader, model_cnn, opt)
    test_err, test_loss = epoch(test_loader, model_cnn) 

    print(*("{:.6f}".format(i) for i in (train_err, test_err)), sep="\t")
    #print(train_err)



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0.728640	0.611100
0.549300	0.472200
0.447660	0.403000
0.391320	0.355400
0.347360	0.310300
0.308760	0.285900
0.276880	0.258600
0.250100	0.239700
0.226280	0.239100
0.211860	0.237200
0.196280	0.226400
0.179620	0.209500
0.166400	0.215800
0.155080	0.194200
0.141880	0.192200
0.134940	0.197100
0.121520	0.190200
0.114860	0.187900
0.104300	0.191300
0.098040	0.185800
0.093440	0.191800
0.083460	0.195500
0.081020	0.184100
0.072860	0.190800
0.067600	0.188300
0.064300	0.184800
0.060740	0.190100
0.056960	0.185900
0.055520	0.183100
0.052420	0.189200
0.048400	0.186400
0.046600	0.182300
0.045100	0.187100
0.043460	0.187800
0.041900	0.181800
0.041320	0.186800
0.038880	0.188700
0.038620	0.184300
0.035760	0.187400
0.034360	0.188100
0.036920	0.186600
0.033280	0.186400
0.032220	0.185300
0.031760	0.191400
0.031820	0.191600
0.031480	0.186100
0.029780	0.194600
0.030960	0.187300
0.029280	0.186800
0.027440	0.184200


In [6]:
model_cnn.eval()
torch.save(model_cnn.state_dict(), "TENET_CIFAR10_model_cnn_teacher_T_100.pt")

model_cnn.load_state_dict(torch.load("TENET_CIFAR10_model_cnn_teacher_T_100.pt"))
model_cnn.eval()

Model_Drop(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
  (drop_layer_conv): Dropout2d(p=0.5, inplace=False)
)

In [7]:
torch.save(model_cnn.state_dict(), "/content/gdrive/MyDrive/TENET_CIFAR10_model_cnn_teacher_T_100.pt")